In [1]:
# ! wget https://github.com/alexeygrigorev/minsearch/raw/refs/heads/main/minsearch.py

--2025-06-14 17:34:29--  https://github.com/alexeygrigorev/minsearch/raw/refs/heads/main/minsearch.py
Resolving github.com (github.com)... 140.82.116.4
Connecting to github.com (github.com)|140.82.116.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch.py [following]
--2025-06-14 17:34:29--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4073 (4.0K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.98K  --.-KB/s    in 0s      

2025-06-14 17:34:29 (35.7 MB/s) - ‘minsearch.py’ saved [4073/4073]



In [2]:
# import minsearch

/workspaces/llm-zoomcamp-2025/01-intro/minsearch.py:10: UserWarning: Now minsearch is installable via pip: 'pip install minsearch'. Remove the downloaded file and re-install it with pip.
  warnings.warn(


In [3]:
! pip install minsearch

In [4]:
import minsearch

In [5]:
import json

In [6]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [13]:
# docs_raw

In [14]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [15]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [16]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

SELECT * WHERE course = 'data-engineering-zoomcamp';

In [17]:
q = 'the course has already started, can I still enroll?'

In [18]:
index.fit(documents)

In [19]:
import os
import sys
from dotenv import load_dotenv

from litellm import completion
from typing import List, Dict

from openai import OpenAI

In [20]:
# Load environment variables and set API key
load_dotenv()

True

In [21]:
client = OpenAI()

In [29]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": q}]
)

response.choices[0].message.content

"Whether you can still enroll in a course after it has started depends on the specific institution and course policies. Here are some steps you can take to find out:\n\n1. **Contact the Instructor or Department**: Reach out to the course instructor or the relevant department to inquire about late enrollment. They may make exceptions or have specific policies in place for latecomers.\n\n2. **Check the Institution's Add/Drop Policy**: Most educational institutions have an add/drop period at the beginning of the term where students can register for courses without penalty. Check if this period is still open.\n\n3. **Online Portal**: If your institution has an online registration system, check it for any options to register for the course or to see if spots are still available.\n\n4. **Academic Advisor**: Contact your academic advisor for guidance on enrolling in courses after they have started. They may provide alternative solutions or help navigate the institutional policies.\n\n5. **Wai

In [31]:
# Using LiteLLM

messages=[{"role": "user", "content": q}]

response = completion(
    model="openai/gpt-4o",
    messages=messages,
    # max_tokens=1024
)

response.choices[0].message.content

"It depends on the course and the institution offering it. Some courses have rolling admissions or late enrollment periods, allowing students to join after the start date. Others may not allow late enrollment due to course structure or capacity limits. It's best to contact the institution or check their website to understand their policies on late enrollment. Be sure to inquire about any potential penalties or missed material if you are allowed to join late."

In [32]:
boost = {'question': 3.0, 'section': 0.5}

results = index.search(
    query=q,
    boost_dict=boost,
    num_results=5
)

In [33]:
results

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the cour